# How to Import a Jupyter Notebook from Another

Execute all cells below to import Jupyter notebooks. Then, in a new cell of this document in the same directory location, you could do something such as:

```
from fwdimaging import simulate_set
```

As you do not wish to have all these cells below as header of your notebooks, say to `path` variable (line 11 in `find_notebook()`) where are your Jupyter modules, download this notebook as a Python source code (**File**&rarr;**Download as**&rarr;**Python (.py)**), move it to `~/.ipython/profile_default/startup/` and restart the notebook kernel (**Kernel**&rarr;**Restart**). If the directory does not exist, execute from the command line and try the procedure again:

```
$ ipython profile create
```

This notebook is based on the [documentation](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Importing%20Notebooks.html) created by [Jupyter Team](https://jupyter.org).

In [ ]:
import io, os, sys, types

In [ ]:
import nbformat

from IPython import get_ipython
from IPython.core.interactiveshell import InteractiveShell

In [ ]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        # In path ('list'), put the location path of the folder with your own Jupyter modules.
        path = [os.path.join(os.path.expanduser('~'),
                             'super-scanner-software-s3',
                             'jupyter-modules'), ]
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [ ]:
class NotebookLoader(object):
    """Module Loader for IPython Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing notebook from %s" % path)
                                       
        # load the notebook object
        nb = nbformat.read(path, as_version=4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [ ]:
class NotebookFinder(object):
    """Module finder that locates IPython Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [ ]:
sys.meta_path.append(NotebookFinder())